In [1]:
import os, glob
import json
from functions import load_txt_as_lst, read_alignments

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# Run labse on every sents file in input directory, then write to scratch

In [17]:
sents_dir = "/home/craig.car/repos/chiron/chironata/data/src_data/"
for sents_path in glob.iglob(sents_dir+"*.sents"):
    print(os.path.splitext(os.path.basename(sents_path))[0])
    break

urn:cts:greekLit:tlg0001.tlg001


In [18]:
sents_dir = "/home/craig.car/repos/chiron/chironata/data/french_trans-dev/"
for sents_path in glob.iglob(sents_dir+"*.sents"):
    print(os.path.splitext(os.path.basename(sents_path))[0])
    break

aeschylus_1863


In [24]:
sents_dir = "/home/craig.car/repos/chiron/chironata/data/src_data/"
dir_out = "/scratch/craig.car/sents_embeds/src_embeds/"
    
for sents_path in glob.iglob(sents_dir+"*.sents"):
    filename = os.path.splitext(os.path.basename(sents_path))[0]
    if os.path.isfile(dir_out+filename+".emb") == False:
        print(filename)
        txt_lst = load_txt_as_lst(sents_path)
        # labse_embeds = build_embeddings_huggingface(txt_lst, model)
        print("built embeds")
        path_out = dir_out+filename+".emb"
        # write_to_file(labse_embeds,path_out)
        print(path_out)
    break

# Run sentence retrieval and score
TODO: error with score computing

In [2]:
def compute_sim_matrix_cosine(src_embeds, tgt_embeds):
    # using cosine sim
    return cosine_similarity(src_embeds, tgt_embeds)

def build__src_2_tgt_dict(alignments_lst):
    '''
    If alignment is null on one side, inserts "null" 
    '''
    src_id_to_tgt_ids = defaultdict(set)
    for src, tgt in alignments_lst:
        if (src, tgt) == ([], []):
            continue
        else:
            if src == []:
                src = ["null"]
            if tgt == []:
                tgt = ["null"]
            for src_id in src:
                for tgt_id in tgt:
                    src_id_to_tgt_ids[src_id].add(tgt_id)
    # convert to vals to list for writing to json
    for key in src_id_to_tgt_ids.keys():
        src_id_to_tgt_ids[key] = list(src_id_to_tgt_ids[key])
    return src_id_to_tgt_ids

In [3]:
# load lookup table
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

In [4]:
fr_embed_dir = "/scratch/craig.car/sents_embeds/fr_embeds/"
de_embed_dir = "/scratch/craig.car/sents_embeds/de_embeds/"
en_embed_dir = "/scratch/craig.car/sents_embeds/en_embeds/"
it_embed_dir = "/scratch/craig.car/sents_embeds/it_embeds/"
align_dir = "/home/craig.car/repos/chiron/chironata/data/alignments_rslts/"

dim = 768

src_embeds_dir = "/scratch/craig.car/sents_embeds/src_embeds/"
corpus_len = len([name for name in os.listdir(src_embeds_dir) if os.path.isfile(os.path.join(src_embeds_dir, name))])
print(corpus_len)

709


In [13]:
src_embeds_dir = "/scratch/craig.car/sents_embeds/src_embeds/"
corpus_len = len([name for name in os.listdir(src_embeds_dir) if os.path.isfile(os.path.join(src_embeds_dir, name))])
corpus_top1 = 0
corpus_top5 = 0
corpus_top10 = 0

for src_embed_path in glob.iglob(src_embeds_dir+"*.emb"):
    print(f"working on {src_embed_path}")
    text_top1 = 0
    text_top5 = 0
    text_top10 = 0
    # get embeds and resize
    src_embeddings = np.fromfile(src_embed_path, dtype=np.float32, count=-1)
    src_embeddings.resize(src_embeddings.shape[0] // dim, dim)
    # print(src_embeddings.shape)
    # get translations
    ctsurn = os.path.splitext(os.path.basename(src_embed_path))[0]
    print(ctsurn)
    translations = lookup[ctsurn]
    print(translations)
    for tgt_text in translations:
        print(f"working on {tgt_text} translation")
        # count scores per translation
        top_1 = 0
        top_5 = 0
        top_10 = 0
        num_nulls = 0
        # find embed file
        if os.path.isfile(de_embed_dir+tgt_text+".emb"):
            tgt_embed = de_embed_dir+tgt_text+".emb"
        elif os.path.isfile(en_embed_dir+tgt_text+".emb"):
            tgt_embed = en_embed_dir+tgt_text+".emb"
        elif os.path.isfile(it_embed_dir+tgt_text+".emb"):
            tgt_embed = it_embed_dir+tgt_text+".emb"
        elif os.path.isfile(fr_embed_dir+tgt_text+".emb"):
            tgt_embed = fr_embed_dir+tgt_text+".emb"
        # get tgt embeds
        tgt_embeddings = np.fromfile(tgt_embed, dtype=np.float32, count=-1)
        tgt_embeddings.resize(tgt_embeddings.shape[0] // dim, dim)
        # print(tgt_embeddings.shape)
        # build sim matrix
        sim_matrix = compute_sim_matrix_cosine(src_embeddings, tgt_embeddings)
        print(f"built sim matrix with shape {sim_matrix.shape}")
        # load vec rslts
        align_preds_path = align_dir+ctsurn+"_"+tgt_text+".rslts"
        # print(align_preds_path)
        if os.path.isfile(align_preds_path):
            align_preds = read_alignments(align_preds_path)
        # print(len(align_preds))
        # iterate through src sents, compare sim score (src_sent,pred_sent) to top 1,5,10 scores
        for src_idx, src_sent_vector in enumerate(sim_matrix):
            src2tgt = build__src_2_tgt_dict(align_preds)
            pred_sents_idx = src2tgt[src_idx]
            # sort vector of sim scores
            sorted_sim_vec = np.flip(np.sort(src_sent_vector))
            # check for each pred_sent in pred_sents
            for tgt_sent_idx in pred_sents_idx:
                if tgt_sent_idx == "null":
                    num_nulls += 1
                    continue
                tgt_sent_simscore = sorted_sim_vec[tgt_sent_idx]
                if tgt_sent_simscore >= sorted_sim_vec[0]:
                    top_1 += 1
                    top_5 += 1
                    top_10 += 1
                    continue
                elif tgt_sent_simscore >= sorted_sim_vec[4]:
                    top_5 += 1
                    top_10 += 1
                    continue    
                elif tgt_sent_simscore >= sorted_sim_vec[9]:
                    top_10 += 1
                    print("have top 10")
                    continue
            
            # how many src sents have a pred that's in top 1, top 5, top 10?
            top_1_transl = top_1/(sim_matrix.shape[0] - num_nulls)
            top_5_transl = top_5/(sim_matrix.shape[0] - num_nulls)
            top_10_transl = top_10/(sim_matrix.shape[0] - num_nulls)
            print(f"top 10 is {top_10}")
            print(f"top 10 by transl is {top_10_transl}")
            break
        
        # add to text-level average
        text_top1 += top_1_transl/len(translations)
        text_top5 += top_5_transl/len(translations)
        text_top10 += top_10_transl/len(translations)
        print(f"text top 10 is {text_top10}")
    
    # add to corpus-level average
    corpus_top1 += text_top1/corpus_len
    corpus_top5 += text_top5/corpus_len
    corpus_top10 += text_top10/corpus_len
print(f"top10 corpus is {corpus_top10}")
        


working on /scratch/craig.car/sents_embeds/src_embeds/urn:cts:greekLit:tlg0001.tlg001.emb
urn:cts:greekLit:tlg0001.tlg001
['apolloniusRhodius_1_1791', 'apollonius_1889', 'apolloniusRhodius_1892']
working on apolloniusRhodius_1_1791 translation
built sim matrix with shape (1738, 2980)
top 10 is 2
top 10 by transl is 0.0011507479861910242
text top 10 is 0.0003835826620636747
working on apollonius_1889 translation
built sim matrix with shape (1738, 3247)
have top 10
have top 10
top 10 is 7
top 10 by transl is 0.004027617951668585
text top 10 is 0.0017261219792865364
working on apolloniusRhodius_1892 translation
built sim matrix with shape (1738, 12099)
top 10 is 0
top 10 by transl is 0.0
text top 10 is 0.0017261219792865364
working on /scratch/craig.car/sents_embeds/src_embeds/urn:cts:greekLit:tlg0003.tlg001.emb
urn:cts:greekLit:tlg0003.tlg001
['thucydides_1_1869', 'thucydides_2_1917', 'thucydides_1_1917', 'thucydides_1_1852', 'thucydides_2_1902', 'thucydides_1863', 'thucydides_1_1864']
w

Exception: Failed to parse line "[]:[173]:0.000000"